In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 32054832
paper_name = 'chen_petranovic_2020' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [4]:
datasets.set_index('pmid', inplace=True)

# Load & process the data

In [5]:
original_data1 = pd.read_excel('raw_data/complete score data for SGA.xlsx', sheet_name='nonessential repeat 1')
original_data2 = pd.read_excel('raw_data/complete score data for SGA.xlsx', sheet_name='nonessential repeat 2')

In [6]:
print('Original data dimensions: %d x %d' % (original_data1.shape))
print('Original data dimensions: %d x %d' % (original_data2.shape))

Original data dimensions: 4322 x 4
Original data dimensions: 4322 x 4


In [7]:
original_data1['orfs'] = original_data1['Array ORF'].astype(str)
original_data2['orfs'] = original_data2['Array ORF'].astype(str)

In [8]:
# Eliminate all white spaces & capitalize
original_data1['orfs'] = clean_orf(original_data1['orfs'])
original_data2['orfs'] = clean_orf(original_data2['orfs'])

In [9]:
# Translate to ORFs 
original_data1['orfs'] = translate_sc(original_data1['orfs'], to='orf')
original_data2['orfs'] = translate_sc(original_data2['orfs'], to='orf')

In [10]:
# Make sure everything translated ok
t = looks_like_orf(original_data1['orfs'])
print(original_data1.loc[~t,])

Empty DataFrame
Columns: [Query Name, Array ORF, Array Name, Score, orfs]
Index: []


In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data2['orfs'])
print(original_data2.loc[~t,])

Empty DataFrame
Columns: [Query Name, Array ORF, Array Name, Score, orfs]
Index: []


In [12]:
data1 = original_data1[['orfs','Score']].groupby('orfs').mean()
data2 = original_data2[['orfs','Score']].groupby('orfs').mean()

In [13]:
data = data1.join(data2, lsuffix='_1', rsuffix='_2')

In [14]:
data['data'] = data[['Score_1','Score_2']].mean(axis=1)

In [15]:
data = data.drop(columns=['Score_1','Score_2'])

In [17]:
data.index.name='orf'

# Prepare the final dataset

In [18]:
dataset_ids = [16408]
datasets = datasets.reindex(index=dataset_ids)

In [19]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [20]:
print('Final data dimensions: %d x %d' % (data.shape))

Final data dimensions: 4268 x 1


## Subset to the genes currently in SGD

In [21]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [22]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

# Normalize

In [23]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [24]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [25]:
data_norm[data.isnull()] = np.nan

In [26]:
data_all = data.join(data_norm)

In [27]:
data_all.head()

dataset_id          16408          
data_type           value    valuez
gene_id orf                        
2       YAL002W  0.045350  0.161340
1863    YAL004W -0.049960 -0.280535
4       YAL005C  0.016845  0.029186
5       YAL007C  0.187275  0.819331
6       YAL008W  0.257400  1.144444

# Print out

In [28]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [29]:
from IO.save_data_to_db3 import *

In [30]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 32054832...
Inserting the new data...


100%|██████████| 1/1 [00:05<00:00,  5.88s/it]

Updating the data_modified_on field...
